In [1]:
%env SAGE_MACAULAY2_COMMAND /opt/homebrew/bin/M2

env: SAGE_MACAULAY2_COMMAND=/opt/homebrew/bin/M2


In [3]:
# Given a group G as GAP input
# Returns a pair (A(G), genDisc) where 
#       - A(G) is the Burnside Ring of G (as a SAGE object)
#       - genDisc is a dictionary of structure descriptions of the generators
#       - note that this structure description does not distinguish between distinct but abstractly isomorphic subgroups
def burnside_ring(G):
    tomG = G.TableOfMarks()
    matG = Matrix(ZZ, tomG.MatTom())
    nG = matG.dimensions()[0]
    repsG = [tomG.RepresentativeTom(n+1).StructureDescription() for n in range(nG)]
    baseRing = PolynomialRing(ZZ,'x',nG)
    baseGens = baseRing.gens()
    index_of_trivial = nG-1 # GAP always put the trivial orbit at the end of the list
    relations = [baseGens[index_of_trivial]-1]
    for i in range(nG):
        for j in range(i,nG):
            relations.append(baseGens[i]*baseGens[j] - sum(coeff * baseGens[k] for k,coeff in enumerate(matG.solve_left(vector(ZZ, [matG[i][k] * matG[j][k] for k in range(nG)])))))
    burnsideRing = baseRing.quotient(relations)
    generatorDescriptions = { baseGens[i] : str(repsG[index_of_trivial]) + "/" + str(repsG[i]) for i in range(nG) }
    return (burnsideRing,generatorDescriptions)

def print_minimal_presentation(ring):
    print(macaulay2(ring).minimalPresentation())

In [3]:
(AC2,genDisc) = burnside_ring(gap("SymmetricGroup(2)"))
print_minimal_presentation(AC2)

  ZZ[x0]
---------
  2
x0  - 2x0


# Computing norms in Burnside Tambara Functors

In [21]:
# norm from H to G
def burnside_norm(H, G, vec):
    tomH = H.TableOfMarks()
    matH = matrix(ZZ, tomH.MatTom())
    nH = matH.dimensions()[0]
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)]
    tomG = G.TableOfMarks()
    matG = matrix(ZZ, tomG.MatTom())
    nG = matG.dimensions()[0]
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)]
    marks_in = vec * matH
    marks_out = vector(ZZ, [1]*nG)
    for i in range(nG):
        for (g, _) in  G.DoubleCosetRepsAndSizes(repsG[i], H):
            idx = next(j for j,r in enumerate(repsH) if H.IsConjugate(r, gap.Intersection(H, repsG[i].ConjugateGroup(g))))
            marks_out[i] *= marks_in[idx]
    return matG.solve_left(marks_out)

In [20]:
H = gap("SymmetricGroup(4)")
G = gap("SymmetricGroup(5)")
vec = vector(ZZ, 11, {3:1, 4:1})
tomH = H.TableOfMarks()
matH = matrix(ZZ, tomH.MatTom())
nH = matH.dimensions()[0]
repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)]
tomG = G.TableOfMarks()
matG = matrix(ZZ, tomG.MatTom())
nG = matG.dimensions()[0]
repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)]

def mark_to_str_G(v):
    return " + ".join(f"{c}[{G.StructureDescription()}/{repsG[i].StructureDescription()}]" for i,c in enumerate(v) if c != 0)

def mark_to_str_H(v):
    return " + ".join(f"{c}[{H.StructureDescription()}/{repsH[i].StructureDescription()}]" for i,c in enumerate(v) if c != 0)

marks_in = vec * matH
marks_out = vector(ZZ, [1]*nG)
for i in range(nG):
    for (g, _) in  G.DoubleCosetRepsAndSizes(repsG[i], H):
        idx = next(j for j,r in enumerate(repsH) if H.IsConjugate(r, gap.Intersection(H, repsG[i].ConjugateGroup(g))))
        marks_out[i] *= marks_in[idx]
nm = matG.solve_left(marks_out)

print("nm_{S_4}^{S_5}(" + mark_to_str_H(vec) + ") = " + mark_to_str_G(nm))

nm_{S_4}^{S_5}(1[S4/C3] + 1[S4/C2 x C2]) = 4341[S5/1] + 264[S5/C2] + 11[S5/C3] + 14[S5/C2 x C2] + 2[S5/C5] + 6[S5/S3] + 3[S5/D10]


In [2]:
def multiplicative_cohomologicalization(G):
    tomG = G.TableOfMarks()
    matG = matrix(ZZ, tomG.MatTom())
    nG = matG.dimensions()[0]
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)]
    baseRing = PolynomialRing(ZZ,'x',nG)
    baseGens = baseRing.gens()
    index_of_trivial = nG-1 # GAP always put the trivial orbit at the end of the list
    relations = [baseGens[index_of_trivial]-1]
    for i in range(nG):
        for j in range(i+1,nG):
            relations.append(baseGens[i]*baseGens[j] - sum(coeff * baseGens[k] for k,coeff in enumerate(matG.solve_left(vector(ZZ, [matG[i][k] * matG[j][k] for k in range(nG)])))))
    # set nm_L^G([G:K]) = [G/K]^[G:L] for all K,L
    for i in range(nG):# index of K
        for j,L in enumerate(repsG):
            tomL = L.TableOfMarks()
            nL = int(tomL.OrdersTom().Length())
            repsL = [tomL.RepresentativeTom(n+1) for n in range(nL)]
            marks_in = vector(ZZ, [matG[i][0]]*nL)
            marks_out = vector(ZZ, [1]*nG)
            for m,H in enumerate(repsG):
                for (g, _) in  G.DoubleCosetRepsAndSizes(H,L):
                    idx = next(s for s,r in enumerate(repsL) if L.IsConjugate(r, gap.Intersection(L, H.ConjugateGroup(g))))
                    marks_out[m] *= marks_in[idx]
            nm_vec = matG.solve_left(marks_out)
            relations.append(baseGens[i]**matG[j][0] - sum(coeff * baseGens[k] for k,coeff in enumerate(nm_vec)))
    return baseRing.quotient(relations)

In [ ]:
G = gap("AlternatingGroup(5)")
A_G_coh = multiplicative_cohomologicalization(G)
print_minimal_presentation(A_G_coh)